In [94]:
import pandas as pd
import numpy as np
import tensorflow as tf
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split
from transformers import PreTrainedTokenizerBase
from datasets import Dataset


In [154]:
data = pd.read_csv('/Users/kristian/Documents/Skole/7. semester/NLP/Exam/ASD_classification/CHI_data.csv')


In [155]:
def preprocess_tokens(df):

    # Remove punctuations and numbers
    df['tokens2'] = df['tokens2'].str.replace('[^a-zA-Z]', ' ', regex=True)

    # Single character removal
    df['tokens2'] = df['tokens2'].str.replace(r"\s+[a-zA-Z]\s+", ' ', regex=True)

    # Removing multiple spaces
    df['tokens2'] = df['tokens2'].str.replace(r'\s+', ' ', regex=True)

    # Drop spaces created when deleting single period values
    df = df[df.tokens2 != ' ']

    return df

# use function on data frame
data = preprocess_tokens(data)

In [158]:
data

,Unnamed: 0,participant,age,id,group,tokens2
6,110,CHI,45.633333,11312/a-00032743-1,TD,cow
12,135,CHI,45.633333,11312/a-00032743-1,TD,tree
13,141,CHI,45.633333,11312/a-00032743-1,TD,bandaid
14,147,CHI,45.633333,11312/a-00032743-1,TD,ow ipep
17,162,CHI,45.633333,11312/a-00032743-1,TD,brush
...,...,...,...,...,...,...
8347,300,CHI,28.800000,11312/a-00005262-1,TD,love you haircut
8348,304,CHI,28.800000,11312/a-00005262-1,TD,no Mama no not just ponytail
8349,306,CHI,28.800000,11312/a-00005262-1,TD,I just your hair
8350,310,CHI,28.800000,11312/a-00005262-1,TD,haircut


In [9]:
model_checkpoint = "distilbert-base-uncased"

In [159]:
label_list = ['TD', 'ASD']

In [62]:
# Split dataset into train, test, val (80, 10, 10)
train, test = train_test_split(data, test_size=0.1)
train, val = train_test_split(train, test_size=0.1)

In [63]:
# Create new index with the same length as the splits
train_idx = [i for i in range(len(train.index))]
test_idx = [i for i in range(len(test.index))]
val_idx = [i for i in range(len(val.index))]


In [65]:
# Convert to numpy
x_train = train['tokens2'].values[train_idx]
x_test = test['tokens2'].values[test_idx]
x_val = val['tokens2'].values[val_idx]


In [72]:
# convert y splits to numpy
y_train = train['group'].values[train_idx]
y_test = test['group'].values[test_idx]
y_val = val['group'].values[val_idx]

In [160]:
x_train

array(['yeah ', 'yes ', 'pink pink telephone ki ', ..., 'on plate ',
       'Goldilocks ', 'swim and play Ring Around The Rosy '], dtype=object)

In [148]:
# Create tokenizer from pretrained model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Tokenize datasets
train_tok = tokenizer(list(x_train), truncation=True, padding=True, max_length=20)
val_tok = tokenizer(list(x_val), truncation=True, padding=True, max_length=20)
test_tok = tokenizer(list(x_test),  truncation=True, padding=True, max_length=20)

# label
#train_tok["labels"] = train_tok["input_ids"].copy()
#val_tok["labels"] = val_tok["input_ids"].copy()
#test_tok["labels"] = test_tok["input_ids"].copy()


In [135]:
# Make datasets
train_dataset = Dataset.from_dict(train_tok)

val_dataset = Dataset.from_dict(val_tok)

test_dataset = Dataset.from_dict(test_tok)


In [161]:
train_dataset


Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 5762
})

In [129]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t